In [36]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
commission = 5
buy_times = 0
absolute_profit = 0
absolute_loss = 0

In [37]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


In [38]:
symbols = """
select distinct(name) from stockdatamonthly order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [39]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [40]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = None
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    training = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        self.training = list()

sql1= """
select distinct(name) from stockdatamonthly order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)



<h1> Updating resources

In [41]:
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdatamonthly where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done
100 % done

In [42]:
sql_dates = """
select distinct(date) from stockdatamonthly order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Trading strategy with the training sets generation

In [43]:

def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global buy_times
    global absolute_loss
    global absolute_profit
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdatamonthly where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    
                    
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        
                        current_symbol.bought = current_price
                        current_symbol.stop_order = current_symbol.support * 0.97
                        quantity = (0.02 * capital) / current_price
                        remaining_cash = remaining_cash - commission - (quantity * current_price)                        
                        position[symbol]=quantity
                        current_symbol.times_bought += 1
                        print("Buying ", quantity, "stocks of ", symbol, "At price", current_symbol.bought, "on date", date)
                        training_inst = training_set_extractor(data_till_date, date)                            
                        current_symbol.training.append(training_inst)
                        
                            
                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                current_symbol.training[-1][-1] = 1
                                win += 1
                                current_symbol.flag_for_wins = True
                        
                        
                    if current_symbol.stop_order > current_price:
                        
                        
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        if profit_today > 0:
                            current_symbol.training[-1][-1]= 1
                            if current_symbol.flag_for_wins == False:
                                win += 1
                            current_symbol.flag_for_wins = False
                            absolute_profit += profit_today
                        else:
                            loss += 1
                            current_symbol.training[-1][-1] = 0
                            current_symbol.flag_for_wins = False
                            absolute_loss += profit_today
                        profit += profit_today
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("Profit",profit)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [44]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def training_set_extractor(data, date):
    past_five_months = data.tail(5)
    difference = abs(len(past_five_months['2. high'].tolist()) - 5)
    zeros_list = [0] * (difference)
    training_instance = [zeros_list[:], zeros_list[:], 'na']
        
    training_instance[0].extend(past_five_months['2. high'].tolist()[:])
    training_instance[1].extend(past_five_months['5. volume'].tolist()[:])
    return training_instance[:]
   

In [45]:
automatic_trader(symbol_list)

Buying  5.194805194805195 stocks of  AEP At price 38.5 on date 2000-04-28
Buying  2.188662727073758 stocks of  AES At price 91.38 on date 2000-04-28
Buying  9.090909090909092 stocks of  ALB At price 22.0 on date 2000-04-28
Buying  6.0606060606060606 stocks of  APD At price 33.0 on date 2000-04-28
Buying  6.516780710329098 stocks of  ASB At price 30.69 on date 2000-04-28
Buying  6.583278472679394 stocks of  ASTE At price 30.38 on date 2000-04-28
Buying  7.017543859649122 stocks of  ATR At price 28.5 on date 2000-04-28
Buying  12.121212121212121 stocks of  AXL At price 16.5 on date 2000-04-28
Buying  3.536067892503536 stocks of  BAC At price 56.56 on date 2000-04-28
Buying  3.007518796992481 stocks of  BAX At price 66.5 on date 2000-04-28
Buying  11.11111111111111 stocks of  BDN At price 18.0 on date 2000-04-28
Buying  5.350454788657036 stocks of  BMI At price 37.38 on date 2000-04-28
Buying  8.488964346349746 stocks of  BNS At price 23.56 on date 2000-04-28
Buying  9.38526513374003 stoc

Buying  10.38961038961039 stocks of  MGEE At price 19.25 on date 2000-05-31
Buying  1.7777777777777777 stocks of  MMC At price 112.5 on date 2000-05-31
Buying  37.17472118959108 stocks of  MNR At price 5.38 on date 2000-05-31
Buying  11.267605633802816 stocks of  MOS At price 17.75 on date 2000-05-31
Buying  8.602150537634408 stocks of  MSM At price 23.25 on date 2000-05-31
Buying  4.584001833600733 stocks of  NBR At price 43.63 on date 2000-05-31
Buying  4.4503782821539835 stocks of  NE At price 44.94 on date 2000-05-31
Buying  8.24742268041237 stocks of  O At price 24.25 on date 2000-05-31
Buying  2.0 stocks of  PGR At price 100.0 on date 2000-05-31
Buying  31.68065895770632 stocks of  PKD At price 6.313 on date 2000-05-31
Buying  15.686274509803921 stocks of  PKG At price 12.75 on date 2000-05-31
Buying  2.185792349726776 stocks of  PLXS At price 91.5 on date 2000-05-31
Buying  8.554319931565441 stocks of  PSA At price 23.38 on date 2000-05-31
Buying  8.51063829787234 stocks of  RCI

Buying  30.76923076923077 stocks of  ASTC At price 6.5 on date 2000-07-31
Buying  3.9800995024875623 stocks of  BA At price 50.25 on date 2000-07-31
Buying  3.1746031746031744 stocks of  CAMP At price 63.0 on date 2000-07-31
Buying  5.298013245033113 stocks of  COLM At price 37.75 on date 2000-07-31
Buying  6.4 stocks of  CPT At price 31.25 on date 2000-07-31
Buying  6.4 stocks of  CRS At price 31.25 on date 2000-07-31
Buying  5.714285714285714 stocks of  CTWS At price 35.0 on date 2000-07-31
Buying  2.1220159151193636 stocks of  DB At price 94.25 on date 2000-07-31
Selling  8.019246190858059 stocks of DCI  at price 20.5 on date 2000-07-31
Buying  16.93480101608806 stocks of  DNI At price 11.81 on date 2000-07-31
Buying  26.44453259288642 stocks of  DSU At price 7.563 on date 2000-07-31
Buying  444.44444444444446 stocks of  DVCR At price 0.45 on date 2000-07-31
Buying  20.0 stocks of  EMCI At price 10.0 on date 2000-07-31
Buying  10.922992900054616 stocks of  ESCA At price 18.31 on dat

Buying  15.54001554001554 stocks of  CATO At price 12.87 on date 2000-09-29
Buying  5.128205128205129 stocks of  CET At price 39.0 on date 2000-09-29
Buying  6.165228113440198 stocks of  CHE At price 32.44 on date 2000-09-29
Selling  3.888024883359254 stocks of DIOD  at price 21.88 on date 2000-09-29
Buying  5.7339449541284395 stocks of  DNB At price 34.88 on date 2000-09-29
Buying  12.121212121212121 stocks of  DNBF At price 16.5 on date 2000-09-29
Buying  11.11111111111111 stocks of  DSWL At price 18.0 on date 2000-09-29
Buying  27.11864406779661 stocks of  EEI At price 7.375 on date 2000-09-29
Selling  3.493449781659389 stocks of F  at price 28.5 on date 2000-09-29
Buying  1.3333333333333333 stocks of  FBC At price 150.0 on date 2000-09-29
Buying  2.6490066225165565 stocks of  FCNCA At price 75.5 on date 2000-09-29
Buying  6.476683937823834 stocks of  FMS At price 30.88 on date 2000-09-29
Buying  6.4 stocks of  GBL At price 31.25 on date 2000-09-29
Buying  14.673514306676449 stocks 

Buying  6.25 stocks of  COG At price 32.0 on date 2000-12-29
Buying  12.853470437017995 stocks of  COLB At price 15.56 on date 2000-12-29
Buying  9.195402298850574 stocks of  CSS At price 21.75 on date 2000-12-29
Buying  5.755395683453237 stocks of  DBD At price 34.75 on date 2000-12-29
Selling  1.8433179723502304 stocks of EA  at price 45.5 on date 2000-12-29
Buying  8.143322475570033 stocks of  ES At price 24.56 on date 2000-12-29
Buying  3.463203463203463 stocks of  ESS At price 57.75 on date 2000-12-29
Buying  16.15508885298869 stocks of  EVN At price 12.38 on date 2000-12-29
Buying  8.375209380234507 stocks of  FULT At price 23.88 on date 2000-12-29
Buying  14.545454545454545 stocks of  GLT At price 13.75 on date 2000-12-29
Buying  4.775549188156638 stocks of  HBHC At price 41.88 on date 2000-12-29
Buying  14.035087719298245 stocks of  HDNG At price 14.25 on date 2000-12-29
Buying  5.271481286241434 stocks of  HE At price 37.94 on date 2000-12-29
Buying  9.013068949977466 stocks o

...

Selling  10.309278350515465 stocks of LBDC  at price 18.25 on date 2017-09-29
Buying  2.8368794326241136 stocks of  LIVN At price 70.5 on date 2017-09-29
Buying  1.0797387032338175 stocks of  LRCX At price 185.23 on date 2017-09-29
Buying  12.987012987012987 stocks of  MACK At price 15.4 on date 2017-09-29
Buying  3.0907124092103233 stocks of  MCHI At price 64.71 on date 2017-09-29
Selling  11.049723756906076 stocks of MHLD  at price 8.0 on date 2017-09-29
Buying  3.5505059470974616 stocks of  NBLX At price 56.33 on date 2017-09-29
Buying  7.407407407407407 stocks of  NBN At price 27.0 on date 2017-09-29
Buying  7.249029536170845 stocks of  NUSC At price 27.5899 on date 2017-09-29
Selling  3.328340822100183 stocks of NUVA  at price 62.75 on date 2017-09-29
Buying  1.0460251046025104 stocks of  NVDA At price 191.2 on date 2017-09-29
Buying  5.945303210463734 stocks of  OKTA At price 33.64 on date 2017-09-29
Buying  17.937219730941703 stocks of  OMN At price 11.15 on date 2017-09-29
Sell

Buying  12.987012987012987 stocks of  SPRO At price 15.4 on date 2017-11-30
Selling  2.5198469444965914 stocks of SPTL  at price 36.6501 on date 2017-11-30
Selling  2.262443438914027 stocks of SPYG  at price 32.9889 on date 2017-11-30
Buying  4.433606739082244 stocks of  TNET At price 45.11 on date 2017-11-30
Buying  12.113870381586915 stocks of  TWO At price 16.51 on date 2017-11-30
Buying  1.567459539950625 stocks of  VMW At price 127.595 on date 2017-11-30
Buying  4.937296336526118 stocks of  WING At price 40.508 on date 2017-11-30
Buying  1.997403375611705 stocks of  WMT At price 100.13 on date 2017-11-30
Buying  22.22222222222222 stocks of  YECO At price 9.0 on date 2017-11-30
Buying  11.061946902654869 stocks of  ZTO At price 18.08 on date 2017-11-30
Buying  19.12960306073649 stocks of  INDUU At price 10.455 on date 2017-12-27
Selling  5.425935973955507 stocks of ALEX  at price 29.99 on date 2017-12-29
Selling  22.22222222222222 stocks of AMRH  at price 3.8535 on date 2017-12-29


Buying  4.23728813559322 stocks of  XHB At price 47.2 on date 2018-01-31
Buying  2.1390374331550803 stocks of  XNTK At price 93.5 on date 2018-01-31
Buying  19.801980198019802 stocks of  CMSSU At price 10.1 on date 2018-02-26
Buying  460.8294930875576 stocks of  KBLMR At price 0.434 on date 2018-02-26
Selling  7.139287499107589 stocks of AAOI  at price 34.98 on date 2018-02-28
Selling  6.8119891008174385 stocks of AKR  at price 25.02 on date 2018-02-28
Buying  11.737089201877934 stocks of  AQ At price 17.04 on date 2018-02-28
Selling  54.49591280653951 stocks of BKYI  at price 2.225 on date 2018-02-28
Buying  4.4732721986132855 stocks of  BL At price 44.71 on date 2018-02-28
Buying  3.07739652254193 stocks of  CNMD At price 64.99 on date 2018-02-28
Buying  4.415011037527594 stocks of  COUP At price 45.3 on date 2018-02-28
Selling  11.255367403330462 stocks of CRVS  at price 9.1329 on date 2018-02-28
Selling  7.374631268436578 stocks of CTV  at price 23.11 on date 2018-02-28
Selling  7.

Buying  7.477753682793689 stocks of  ECCY At price 26.746 on date 2018-04-30
Buying  6.722689075630252 stocks of  ENVA At price 29.75 on date 2018-04-30
Selling  9.789525208027412 stocks of ETX  at price 19.11 on date 2018-04-30
Selling  7.243752263672582 stocks of EZT  at price 25.47 on date 2018-04-30
Selling  7.773027594247959 stocks of FBHS  at price 60.34 on date 2018-04-30
Selling  1.6906170752324599 stocks of FISV  at price 72.62 on date 2018-04-30
Selling  9.314022260513203 stocks of GEMP  at price 6.75 on date 2018-04-30
Selling  35.52397868561279 stocks of HDSN  at price 4.97 on date 2018-04-30
Buying  5.019311802158806 stocks of  IHC At price 39.8461 on date 2018-04-30
Buying  2.1468441391155 stocks of  JOBS At price 93.16 on date 2018-04-30
Buying  7.773027594247959 stocks of  KCAPL At price 25.73 on date 2018-04-30
Buying  19.51219512195122 stocks of  LPTX At price 10.25 on date 2018-04-30
Selling  3.2733224222585924 stocks of MATW  at price 52.05 on date 2018-04-30
Buying

Selling  4.444444444444445 stocks of OLED  at price 102.25 on date 2018-06-29
Selling  30.76923076923077 stocks of OMNT  at price 1.129 on date 2018-06-29
Buying  7.251631617113851 stocks of  OMOM At price 27.58 on date 2018-06-29
Buying  19.607843137254903 stocks of  OSPR At price 10.2 on date 2018-06-29
Selling  10.11378002528445 stocks of PAM  at price 51.19 on date 2018-06-29
Buying  7.270083605961468 stocks of  PRFT At price 27.51 on date 2018-06-29
Selling  20.14098690835851 stocks of REPH  at price 5.55 on date 2018-06-29
Buying  7.7429345722028655 stocks of  RPUT At price 25.83 on date 2018-06-29
Buying  37.80718336483932 stocks of  SENS At price 5.29 on date 2018-06-29
Buying  4.2384914361280535 stocks of  SMMD At price 47.1866 on date 2018-06-29
Selling  12.376237623762377 stocks of SUPV  at price 20.31 on date 2018-06-29
Selling  1.232134056185313 stocks of USD  at price 53.22 on date 2018-06-29
Selling  8.00640512409928 stocks of VVV  at price 21.76 on date 2018-06-29
Buyin

<h1> Extracting training sets from the trading strategy's cache

In [46]:
training = []
for i in symbol_list:
    if(stock_equities[i].training):
        if stock_equities[i].training[-1][-1] == 'na':
            del stock_equities[i].training[-1]
        training.extend(stock_equities[i].training)
        
X = []
Y = []


random.shuffle(training)


for i in training:
    X.append([i[0], i[1]])
    Y.append([i[2]])

<h1> Saving training sets 

In [47]:
import pickle
pickle_out = open("Y5Monthly.pickle", "wb")
pickle.dump(Y, pickle_out)
pickle_out.close()


pickle_out = open("X5Monthly.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

In [49]:
len(training)

3387